# Predicting Earthquake Damage in Nepal

**Objective:** Predict severe building damage in the **Kavrepalanchok District** : district_id = 3.

**Data Selection:**
While the source SQL database contains four tables, this analysis focuses on a specific subset of features joined from the following tables:

* **`id_map`**: Selected `vdcmun_id` (Location context).
* **`building_damage`**: Selected `damage_grade` (The target variable).
* **`household_demographics`**: All columns included.
* **`building_structure`**: All columns included.

 ## Wrangling Data with SQL

In [ ]:
import sqlite3
import numpy as  np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from category_encoders import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from category_encoders import OrdinalEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.tree import DecisionTreeClassifier, plot_tree

### Connect

In [ ]:
%reload_ext sql
%sql sqlite:///nepal.sqlite

### Explore Dataset

In [ ]:
%%sql
SELECT
  *
FROM
  sqlite_schema

In [ ]:
%%sql
SELECT
  name
FROM
  sqlite_schema
WHERE
  type = "table"

In [ ]:
%%sql
SELECT
  *
FROM
  id_map
LIMIT
  5

In [ ]:
%%sql
SELECT
  *
FROM
  building_structure
LIMIT
  5

In [ ]:
%%sql
SELECT
  *
FROM
  building_damage
LIMIT
  5

In [ ]:
%%sql
SELECT
  *
FROM
  household_demographics
LIMIT
  5

## Sql query

In [ ]:
%%sql
SELECT
  h.*,
  i.vdcmun_id,
  d.damage_grade,
  bs.*
FROM
  household_demographics AS h
  JOIN building_structure AS bs ON i.building_id = bs.building_id
  JOIN id_map AS i ON i.household_id = h.household_id
  JOIN building_damage AS d ON i.building_id = d.building_id
WHERE
  district_id = 3
LIMIT
  5

### import

In [ ]:
def wrangle(db_path):
    # Connect to database
    conn = sqlite3.connect(db_path)

    # Construct query
   
    query = """
             SELECT h.*,
                    i.vdcmun_id,
                    d.damage_grade,
                    bs.*
          FROM household_demographics AS h
          JOIN building_structure AS bs  ON i.building_id =bs.building_id 
          JOIN id_map AS i ON i.household_id = h.household_id
          JOIN building_damage  AS d ON i.building_id = d.building_id
          WHERE district_id = 3
        
          """
    
    # Read query results into DataFrame
    df = pd.read_sql(query, conn, index_col="household_id")

    # Identify leaky columns
    drop_cols = [col for col in df.columns if "post_eq" in col]

    # Add high-cardinality / redundant column
    drop_cols.append("building_id")

    # Create binary target column
    df["damage_grade"] = df["damage_grade"].str[-1].astype(int)
    df["severe_damage"] = (df["damage_grade"] > 3).astype(int)

    # Drop old target
    drop_cols.append("damage_grade")

    # Drop multicollinearity column
    drop_cols.append("count_floors_pre_eq")
  
    # Group caste column
    top_10 = df["caste_household"].value_counts().head(10).index
    df["caste_household"] = df["caste_household"].apply(lambda c: c if c in top_10 else "Other")
    # Drop columns
    df.drop(columns=drop_cols, inplace=True)

    return df

## Data Wrangling Function

The `wrangle` function acts as our primary ETL (Extract, Transform, Load) pipeline. It handles the connection to the SQLite database, cleans the data, and formats it for the machine learning model.

### 1. Data Extraction (SQL)
We construct a query to join information from four tables to create a comprehensive dataset:
* **`household_demographics`**: Socio-economic data.
* **`building_structure`**: Physical building attributes.
* **`building_damage`**: The target variable (damage grade).
* **`id_map`**: Used to link households to buildings and filter by location.
* **Filter:** We restrict the data to **District 3** to keep the dataset manageable and focused.

### 2. Handling Data Leakage
We identify and drop columns containing `post_eq` (post-earthquake). These columns (e.g., how long reconstruction took) represent information we would not have at the moment of prediction. Including them would cause **data leakage**, giving the model "answers" it shouldn't see.

### 3. Feature Engineering
* **Binary Target:** The original `damage_grade` is a categorical scale (e.g., "Grade 1" to "Grade 5"). We convert this into a binary target named `severe_damage`:
    * **0 (Not Severe):** Grades 1, 2, 3
    * **1 (Severe):** Grades 4, 5
* **Cardinality Reduction:** The `caste_household` column has many unique values. To prevent overfitting, we keep the top 10 most common castes and group the rest into an "Other" category.

### 4. Dimensionality Reduction
We drop high-cardinality identifiers like `building_id` and multicollinear features like `count_floors_pre_eq` to simplify the model and improve performance.

In [ ]:
df = wrangle("nepal.sqlite")
df.head()

## Explore

In [ ]:
# Create correlation matrix
correlation = df.select_dtypes("number").drop(columns="severe_damage").corr()
# Plot heatmap of `correlation`
sns.heatmap(correlation, annot=True);

In [ ]:

plt.figure(figsize=(10, 6))
sns.boxplot(x="severe_damage", y="height_ft_pre_eq", data=df)

plt.xlabel("Severe Damage (0 = No, 1 = Yes)")
plt.ylabel("Building Height (ft)")
plt.title("Distribution of Building Height by Damage Class")
plt.show()

In [ ]:
fig, ax = plt.subplots() 

# Create the Seaborn boxplot 
sns.boxplot(x="severe_damage", y="plinth_area_sq_ft", data=df, ax=ax)

# Set labels and title
ax.set_xlabel("Severe Damage")
ax.set_ylabel("Plinth Area [sq. ft.]")
ax.set_title("Kavrepalanchok, Plinth Area vs Building Damage");

In [ ]:
roof_pivot = pd.pivot_table(
    df, index="roof_type", values="severe_damage", aggfunc=np.mean
).sort_values(by="severe_damage")
roof_pivot

In [ ]:
# Plotting the pivot table
roof_pivot.plot(kind="barh", legend=None)
plt.xlabel("Probability of Severe Damage")
plt.ylabel("Roof Type")
plt.title("Risk of Severe Damage by Roof Type")
plt.axvline(0.5, color="red", linestyle="--", label="50% Risk") # Optional risk line
plt.show()

In [ ]:
foundation_pivot = pd.pivot_table(
    df, index="foundation_type", values="severe_damage", aggfunc=np.mean
).sort_values("severe_damage")
foundation_pivot

In [ ]:


# Plot
foundation_pivot.plot(kind="barh", legend=None)
plt.xlabel("Probability of Severe Damage")
plt.ylabel("Foundation Type")
plt.title("Risk of Severe Damage by Foundation Type")
plt.axvline(0.5, color="red", linestyle="--", label="High Risk Threshold")
plt.show()

## Split

In [ ]:
target = "severe_damage"
X = df.drop(columns=target, axis=1 )
y = df[target]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

## Baseline

In [ ]:
# This block calculates the baseline accuracy
baseline= y_train.value_counts(normalize=True).max()
baseline

## Model building

In [ ]:
# Build model
import time
start_time = time.time()
model = make_pipeline(
    OneHotEncoder(use_cat_names=True),
    LogisticRegression(max_iter=10000)
)
# Fit model to training data
model.fit(X_train, y_train)
end_time = time.time()
print(f"Training took: {round(end_time - start_time, 2)} seconds")

In [ ]:
lr_train_acc = accuracy_score(y_train, model.predict(X_train))
lr_val_acc = accuracy_score(y_test, model.predict(X_test))

print("Logistic Regression, Training Accuracy Score:", lr_train_acc)
print("Logistic Regression, Validation Accuracy Score:", lr_val_acc)

### Feature importance

In [ ]:
features = model.named_steps["onehotencoder"].get_feature_names_out()
importances = model.named_steps["logisticregression"].coef_[0]

In [ ]:
odds_ratios = pd.Series(np.exp(importances), index=features).sort_values()
odds_ratios.head()

In [ ]:
# Horizontal bar chart, five largest coefficients
odds_ratios.tail().plot(kind="barh")

In [ ]:
# Horizontal bar chart, five smallest coefficients
odds_ratios.head().plot(kind="barh")

## Model building with Decision Tree

### Hyperparameters Tuning

In [ ]:
depth_hyperparams = range(1, 16)
training_acc = []
validation_acc = []
for d in depth_hyperparams:
    model_dt = make_pipeline(OrdinalEncoder(), DecisionTreeClassifier(max_depth=d, random_state=42))
    model_dt.fit(X_train, y_train)
    train_acc = accuracy_score(y_train, model_dt.predict(X_train))
    training_acc.append(train_acc)
    val_acc = accuracy_score(y_test, model_dt.predict(X_test))
    validation_acc.append(val_acc)
    

In [ ]:
# Run this cell
submission = pd.Series(validation_acc, index=depth_hyperparams)
submission

In [ ]:
fig, ax = plt.subplots() 

#  Plot the training accuracy on the axes object
ax.plot(depth_hyperparams,training_acc, label="training")

#  Plot the validation accuracy on the same axes object
ax.plot(depth_hyperparams,validation_acc, label="validation") 

#  Set labels and title  
ax.set_xlabel("Max Depth")
ax.set_ylabel("Accuracy Score")
ax.set_title("Validation Curve, Decision Tree Model")

# Add the legend 
ax.legend()

In [ ]:
final_model_dt = make_pipeline(
    OrdinalEncoder(),
    DecisionTreeClassifier(max_depth=11, random_state=42)
)
final_model_dt.fit(X_train, y_train)

In [ ]:
train_acc = accuracy_score(y_train, final_model_dt.predict(X_train))
test_acc = accuracy_score(y_test, final_model_dt.predict(X_test))
print("Decision Tree Training Accuracy:", round(train_acc, 4))
print("Decision Tree Test Accuracy:", round(test_acc, 4))

In [ ]:
# 1. Get feature names from the encoder
features = final_model_dt.named_steps["ordinalencoder"].get_feature_names_out()

# 2. Get importance values from the tree
importances = final_model_dt.named_steps["decisiontreeclassifier"].feature_importances_

# 3. Create a Series and sort it
feat_imp = pd.Series(importances, index=features).sort_values()

# 4. Plot the top 10 most important features
feat_imp.tail(10).plot(kind="barh")
plt.xlabel("Gini Importance")
plt.ylabel("Feature")
plt.title("Feature Importance: What mattered most to the Decision Tree?")
plt.show()

In [ ]:


# 1. Extract the specific model and feature names from the pipeline
classifier = final_model_dt.named_steps["decisiontreeclassifier"]

# 2. Setup the figure size (make it wide so it's readable)
plt.figure(figsize=(25, 12))

# 3. Plot the tree
plot_tree(
    classifier,
    feature_names=features,
    class_names=["Not Severe", "Severe"], 
    filled=True,      
    rounded=True,     
    max_depth=5,      
    fontsize=12
)

plt.title("Decision Tree Flowchart (Top Levels)")
plt.show()

In [85]:
import os

# 1. Create 'images' folder
output_folder = "images"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

print(f"🚀 Saving images to '{output_folder}' folder...\n")

# --- EDA PLOTS ------------------------------------------------

# 1. Correlation Heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation, annot=True, fmt=".2f") # Assuming 'correlation' variable exists
plt.title("Correlation Matrix")
plt.tight_layout()
plt.savefig(os.path.join(output_folder, "eda_heatmap.png"), dpi=150)
plt.close()
print("✅ Saved: eda_heatmap.png")

# 2. Boxplot: Building Height
plt.figure(figsize=(8, 6))
sns.boxplot(x="severe_damage", y="height_ft_pre_eq", data=df)
plt.title("Building Height vs. Severe Damage")
plt.xlabel("Severe Damage (0=No, 1=Yes)")
plt.ylabel("Height (ft)")
plt.savefig(os.path.join(output_folder, "eda_boxplot_height.png"), dpi=150)
plt.close()
print("✅ Saved: eda_boxplot_height.png")

# 3. Bar Chart: Roof Type Risk
plt.figure(figsize=(8, 5))
roof_pivot.plot(kind="barh", legend=None) # Assuming 'roof_pivot' exists
plt.title("Severe Damage Risk by Roof Type")
plt.xlabel("Probability of Damage")
plt.ylabel("Roof Type")
plt.tight_layout()
plt.savefig(os.path.join(output_folder, "eda_roof_risk.png"), dpi=150)
plt.close()
print("✅ Saved: eda_roof_risk.png")

# 4. Bar Chart: Foundation Type Risk
plt.figure(figsize=(8, 5))
foundation_pivot.plot(kind="barh", legend=None)
plt.title("Severe Damage Risk by Foundation Type")
plt.xlabel("Probability of Damage")
plt.ylabel("Foundation Type")
plt.tight_layout()
plt.savefig(os.path.join(output_folder, "eda_foundation_risk.png"), dpi=150)
plt.close()
print("✅ Saved: eda_foundation_risk.png")

# --- MODEL PLOTS ----------------------------------------------

# 5. Logistic Regression: Top 5 Danger Factors
plt.figure(figsize=(10, 5))
odds_ratios.tail().plot(kind="barh")
plt.title("Top 5 Risk Factors (Logistic Regression)")
plt.xlabel("Odds Ratio (>1 increases risk)")
plt.axvline(1, color="red", linestyle="--")
plt.tight_layout()
plt.savefig(os.path.join(output_folder, "model_logreg_danger.png"), dpi=150)
plt.close()
print("✅ Saved: model_logreg_danger.png")

# 6. Logistic Regression: Top 5 Safety Factors
plt.figure(figsize=(10, 5))
odds_ratios.head().plot(kind="barh")
plt.title("Top 5 Safety Factors (Logistic Regression)")
plt.xlabel("Odds Ratio (<1 decreases risk)")
plt.axvline(1, color="red", linestyle="--")
plt.tight_layout()
plt.savefig(os.path.join(output_folder, "model_logreg_safety.png"), dpi=150)
plt.close()
print("✅ Saved: model_logreg_safety.png")

# 7. Decision Tree: Validation Curve
plt.figure(figsize=(10, 6))
plt.plot(depth_hyperparams, training_acc, label="Training Accuracy")
plt.plot(depth_hyperparams, validation_acc, label="Validation Accuracy")
plt.xlabel("Max Depth")
plt.ylabel("Accuracy Score")
plt.title("Validation Curve")
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(output_folder, "model_dt_validation_curve.png"), dpi=150)
plt.close()
print("✅ Saved: model_dt_validation_curve.png")

# 8. Decision Tree: Feature Importance
plt.figure(figsize=(10, 8))
feat_imp.tail(10).plot(kind="barh")
plt.xlabel("Gini Importance")
plt.ylabel("Feature")
plt.title("Decision Tree Feature Importance")
plt.tight_layout()
plt.savefig(os.path.join(output_folder, "model_dt_feature_importance.png"), dpi=150)
plt.close()
print("✅ Saved: model_dt_feature_importance.png")

print("\n🎉 All 8 images saved successfully!")

🚀 Saving images to 'images' folder...

✅ Saved: eda_heatmap.png
✅ Saved: eda_boxplot_height.png
✅ Saved: eda_roof_risk.png
✅ Saved: eda_foundation_risk.png
✅ Saved: model_logreg_danger.png
✅ Saved: model_logreg_safety.png
✅ Saved: model_dt_validation_curve.png
✅ Saved: model_dt_feature_importance.png

🎉 All 8 images saved successfully!


<Figure size 800x500 with 0 Axes>

<Figure size 800x500 with 0 Axes>